# Decision Tree Model – Academic Risk Prediction

This notebook builds an interpretable Decision Tree model to capture non-linear academic risk patterns and improve upon the baseline model.


In [15]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix


In [16]:

data_path="C:/Users/prasad/OneDrive/Documents/student-performance-risk-prediction/data/raw/student-mat.csv"
df=pd.read_csv(data_path,sep=';')

# Create target variable
def assign_risk(grade):
    if grade < 10:
        return "High"
    elif grade < 15:
        return "Medium"
    else:
        return "Low"

df["risk_level"] = df["G3"].apply(assign_risk)

# Remove leakage features
df = df.drop(columns=["G1", "G2", "G3"])


In [17]:
X = df.drop(columns=["risk_level"])
y=df["risk_level"]

In [18]:
numeric_features = X.select_dtypes(include=["int64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_features, categorical_features


(Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
        'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
       dtype='object'),
 Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
        'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'romantic'],
       dtype='object'))

In [19]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_features)
    ],
    remainder="passthrough"
)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [29]:
dt=Pipeline(steps=
    [("preprocessing",preprocessor),
    ("classifier",DecisionTreeClassifier(random_state=42))])

dt.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers con

In [32]:
y_pred=dt.predict(X_test)
print("Classification Report :/n")
print(classification_report(y_test,y_pred))
print("Confusion Matrix:/n")
print(confusion_matrix(y_test,y_pred))

Classification Report :/n
              precision    recall  f1-score   support

        High       0.46      0.42      0.44        26
         Low       0.27      0.20      0.23        15
      Medium       0.48      0.55      0.51        38

    accuracy                           0.44        79
   macro avg       0.40      0.39      0.39        79
weighted avg       0.43      0.44      0.43        79

Confusion Matrix:/n
[[11  2 13]
 [ 2  3 10]
 [11  6 21]]


In [33]:
dt_controll=Pipeline(steps=
    [("preprocessing",preprocessor),
    ("classifier",DecisionTreeClassifier(min_samples_leaf=10,
max_depth=4,random_state=42))])

dt_controll.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers con

In [34]:
y_pred_1=dt_controll.predict(X_test)
print("Classification Report :/n")
print(classification_report(y_test,y_pred_1))
print("Confusion Matrix:/n")
print(confusion_matrix(y_test,y_pred_1))

Classification Report :/n
              precision    recall  f1-score   support

        High       0.45      0.35      0.39        26
         Low       0.33      0.20      0.25        15
      Medium       0.52      0.68      0.59        38

    accuracy                           0.48        79
   macro avg       0.43      0.41      0.41        79
weighted avg       0.46      0.48      0.46        79

Confusion Matrix:/n
[[ 9  2 15]
 [ 3  3  9]
 [ 8  4 26]]


In [35]:
# Get trained decision tree inside pipeline
dt_classifier=dt_controll.named_steps["classifier"]

# Get feature names after preprocessing
feature_names=dt_controll.named_steps["preprocessing"].get_feature_names_out()

#create feature importance dataframe
importance_df=pd.DataFrame({
    "feature":feature_names ,
    "importance":dt_classifier.feature_importances_
        }).sort_values( by="importance" ,ascending=False)



In [36]:
importance_df.head(10)

,feature,importance
31,remainder__failures,0.423358
38,remainder__absences,0.168784
18,cat__schoolsup_yes,0.120537
34,remainder__goout,0.118527
10,cat__Fjob_other,0.090636
27,remainder__Medu,0.078158
0,cat__school_MS,0.000000
6,cat__Mjob_other,0.000000
5,cat__Mjob_health,0.000000
4,cat__Pstatus_T,0.000000


## Feature Importance Analysis

The Decision Tree identifies academic and behavioral factors as the most influential predictors of academic risk.
Past failures, study time, and absences emerge as the strongest indicators, aligning with real-world educational intuition.
Lifestyle factors such as social activity and alcohol consumption also contribute to risk, highlighting the multifaceted nature of student performance.


## Model Explanation (Human-Readable)

The model predicts a student as High Risk when there is a combination of:
- Multiple past academic failures
- Low weekly study time
- High number of absences

Students with consistent study habits and fewer failures are typically classified as Low Risk, even if lifestyle factors vary.


## Example Prediction Explanation

A student was classified as High Risk due to:
- Two or more previous failures
- Study time less than 2–5 hours per week
- Frequent absences

This explanation allows educators to take targeted intervention actions.


In [37]:
import joblib
joblib.dump(dt_controll, "../models/academic_risk_model.pkl")


['../models/academic_risk_model.pkl']